## Notebook Setup and Required Packages

In [1]:
# Data Handling
import pandas as pd
import numpy as np

# IO - getting files and images from MongoDB and S3
from pymongo import MongoClient
from kaggle_secrets import UserSecretsClient
import requests

from concurrent.futures import ThreadPoolExecutor

from pathlib import Path
from PIL import Image
from io import BytesIO

import os
import re
import shutil
import json
import time

# Install speciesnet and related megadetector libraries
!pip install -Uqq speciesnet megadetector-utils

from IPython.display import display
from IPython.display import JSON

from speciesnet import SpeciesNet
import kagglehub



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 31.1 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.7/93.7 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 791.3/791.3 kB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 100.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 100.1 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# Run a quick check to see if the GPU is being used
!python -m speciesnet.scripts.gpu_test

^C
Traceback (most recent call last):
  File "<frozen runpy>", line 189, in _run_module_as_main
  File "<frozen runpy>", line 112, in _get_module_details
  File "/usr/local/lib/python3.11/dist-packages/speciesnet/__init__.py", line 19, in <module>
    from speciesnet.detector import *
  File "/usr/local/lib/python3.11/dist-packages/speciesnet/detector.py", line 31, in <module>
    from yolov5.utils.augmentations import letterbox as yolov5_letterbox
  File "/usr/local/lib/python3.11/dist-packages/yolov5/__init__.py", line 1, in <module>
    from yolov5.helpers import YOLOv5
  File "/usr/local/lib/python3.11/dist-packages/yolov5/helpers.py", line 4, in <module>
    from yolov5.models.common import AutoShape, DetectMultiBackend
  File "/usr/local/lib/python3.11/dist-packages/yolov5/models/common.py", line 18, in <module>
    import cv2
  File "/usr/local/lib/python3.11/dist-packages/cv2/__init__.py", line 181, in <module>
    bootstrap()
  File "/usr/local/lib/python3.11/dist-packages/cv2

In [11]:
# Configuration for Multithreading and Batching
num_batches = 10
max_threads = 8
output_root = Path("output")

# Prepare folders
output_root.mkdir(exist_ok=True)
images_root = Path("images")
images_root.mkdir(exist_ok=True)

## Access The URIs from S3 through MongoDB

In [12]:
# Get the stored mongo uri secret
user_secrets = UserSecretsClient()
mongo_uri = user_secrets.get_secret("MONGO_URI")

# Connect to the MongoDB client
client = MongoClient(mongo_uri)
 
# Access the database and collection
db = client['test']
collection = db['cameratrapmedias'] 
 
# Query the collection to retrieve records with image URLs, metadata, and the first index of 'relativePath'
data = list(collection.aggregate([
    {
        '$project': {
            '_id': 0,
            'publicURL': 1,
            'timestamp': 1,
            'folderName': { '$arrayElemAt': ['$relativePath', 1] },
            'fileName': 1
        }
    },
    # { '$limit': 150 }
]))
 
# Convert the data to a pandas DataFrame for exploration
df = pd.DataFrame(data)

# Export the small array to a CSV file for preview
df.to_csv('ur_test_medias.csv', index=False)

## We are going to create a column that creates a file name to save the image

In [13]:
# This function will format the final string
def make_filename(s):
    # s = s.lower()
    s = re.sub(r'[^\w\s.-]', '', s) # remove special characters except dash or underscore or period
    s = re.sub(r'\s+', '_', s) # replace whitespace with underscore
    return s

# Combine the relative path second (folder name) + fileName
df['imageName'] = df['folderName'] + '--' + df['fileName']
df['imageName'] = df['imageName'].apply(make_filename)

print(df.head())

            timestamp                                          publicURL  \
0 2024-01-27 13:33:15  https://urbanriverrangers.s3.amazonaws.com/ima...   
1 2024-01-24 18:56:50  https://urbanriverrangers.s3.amazonaws.com/ima...   
2 2024-01-24 19:01:54  https://urbanriverrangers.s3.amazonaws.com/ima...   
3 2024-01-24 19:03:05  https://urbanriverrangers.s3.amazonaws.com/ima...   
4 2024-01-24 19:04:19  https://urbanriverrangers.s3.amazonaws.com/ima...   

       fileName                               folderName  \
0  SYFW0060.JPG                   2024-01-30_prologis_02   
1  SYFW0001.JPG  2024-01-30_Learnin_platform_camera_test   
2  SYFW0002.JPG  2024-01-30_Learnin_platform_camera_test   
3  SYFW0004.JPG  2024-01-30_Learnin_platform_camera_test   
4  SYFW0006.JPG  2024-01-30_Learnin_platform_camera_test   

                                           imageName  
0               2024-01-30_prologis_02--SYFW0060.JPG  
1  2024-01-30_Learnin_platform_camera_test--SYFW0...  
2  2024-01-30_Lea

Now that we have a connection to the MongoDB server and access to the URLs, let's use the download images.

## Download Images

In [23]:
%%time
# Create a directory to save the images
output_root.mkdir(exist_ok=True)
path = Path('images')
path.mkdir(exist_ok=True)

# Create a tool for resizing so cropping top and bottom can happen while keeping the aspect ratio
def resize_to_height(image, target_height=256):
    og_width, og_height = image.size
    new_width = int(og_width * (target_height / og_height))
    return image.resize((new_width, target_height))

# Create a tool for downloading and processing images
def process_row(row, dest_folder):
    url = row['publicURL']
    filename = row['imageName']
    # Download the image
    dest = dest_folder/filename

    try:
        # Download image to memory
        response = requests.get(url)
        response.raise_for_status()

        # Open and process the image
        image = Image.open(BytesIO(response.content)).convert("RGB")
        image = resize_to_height(image, target_height=256)
        image.save(dest, format="JPEG", quality=85)
        
    except Exception as e:
        print(f"failed to process{filename}: {e}")

# Download and display some images where at least an animal was found - ex rat
df_test = df[44410:44910] # 500 images with some known animal detections
df_big_chunk = df[0:10000] # first 10000 images

# Process Batches
for batch_idx, df_chunk in enumerate(np.array_split(df_big_chunk, num_batches)): # change to df_test to split a test batch size
    batch_folder = images_root / f'batch_{batch_idx}'
    batch_folder.mkdir(exist_ok=True)

    print(f'Processing batch {batch_idx + 1} / {num_batches} with {len(df_chunk)} images...')

    start = time.time()
    
    with ThreadPoolExecutor(max_workers=max_threads) as executor:
        executor.map(lambda row: process_row(row, batch_folder), [row for _, row in df_chunk.iterrows()])

    end = time.time()
    print(f"Batch {batch_idx+1} took {end - start:.2f} seconds.")
        
print(f'{len(df_big_chunk)} Images Downloaded and Resized')

/usr/local/lib/python3.11/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


Processing batch 1 / 10 with 1000 images...
Batch 1 took 242.91 seconds.
Processing batch 2 / 10 with 1000 images...
Batch 2 took 130.37 seconds.
Processing batch 3 / 10 with 1000 images...
Batch 3 took 213.68 seconds.
Processing batch 4 / 10 with 1000 images...
Batch 4 took 71.11 seconds.
Processing batch 5 / 10 with 1000 images...
Batch 5 took 73.70 seconds.
Processing batch 6 / 10 with 1000 images...
Batch 6 took 86.57 seconds.
Processing batch 7 / 10 with 1000 images...
Batch 7 took 77.40 seconds.
Processing batch 8 / 10 with 1000 images...
Batch 8 took 79.47 seconds.
Processing batch 9 / 10 with 1000 images...
Batch 9 took 77.65 seconds.
Processing batch 10 / 10 with 1000 images...
Batch 10 took 76.38 seconds.
10000 Images Downloaded and Resized
CPU times: user 47min, sys: 3min 19s, total: 50min 20s
Wall time: 18min 49s


In [24]:
# Uncomment and run this if the images need to be redone
# !rm images -r
# !rm docs.zip
# %lsmagic

## Running Species Net on the Full Dataset
Now that we have the max number of images downloaded (19.5GB) let's run speciesnet

Note there might be a better way of doing this using bytes downloaded from s3 - but I haven't figured that part out yet.

### We're going to try a multithreading chunks approach

In [25]:
def print_predictions(predictions_dict: dict) -> None:
    print("Predictions:")
    for prediction in predictions_dict["predictions"][0:1]:
        print(prediction["filepath"], "=>", prediction["prediction"])

### Download Model

In [26]:
# Choose the folder we're going to download the model to
model_path = '/content/models'
os.makedirs(model_path, exist_ok=True)

# Download the model (it will go to a folder like /kaggle/input/...)
download_path = kagglehub.model_download('google/speciesnet/PyTorch/v4.0.1a',
                                          force_download=True)

print('Model downloaded to temporary folder: {}'.format(download_path))

# List the contents of the downloaded directory to identify the actual files/subdirectories
model_files = os.listdir(download_path)

# Copy the contents of the model file to our destination folder
for item_name in model_files:
    source_path = os.path.join(download_path, item_name)
    destination_path = os.path.join(model_path, item_name)
    if os.path.isfile(source_path):
        shutil.copy2(source_path, destination_path)
    elif os.path.isdir(source_path):
        shutil.copytree(source_path, destination_path, dirs_exist_ok=True)

print('{} files copied to: {}'.format(len(model_files),model_path))

Model downloaded to temporary folder: /kaggle/input/speciesnet/pytorch/v4.0.1a/1
6 files copied to: /content/models


In [27]:
# Pick the model we want to use (4.0.1a)
model = SpeciesNet(model_path)

print('Model Loaded')

Model Loaded


In [28]:
# Let's format a request string as a list of dicts (aka JSON string format)
def create_instances(batch_folder):
    image_paths = [f'{batch_folder}/{f}' for f in os.listdir(batch_folder) if f.lower().endswith('.jpg')]

    instances = []
    for image_path in image_paths:
        instances.append({
            'filepath': image_path,
            'latitude': 41.906782,
            'longitude': -87.651927
        })

    # Check that it's saved correctly by verifying the first
    print(instances[0])

    return instances


for batch_index in range(len(os.listdir(images_root))):
    instances = create_instances(f'{images_root}/batch_{batch_index}')

    # make the predictions and get a sense of how long it would take
    %time predictions_dict = model.predict(instances_dict={"instances": instances})

    print_predictions(predictions_dict) # show the first prediction of each batch

    # Save the dict to the batch folder
    with open(f'{images_root}/batch_{batch_index}/predictions_dict_{batch_index}.json', 'w') as f:
        json.dump(predictions_dict, f, indent=2)

    print(f'predictions_dict_{batch_index}.json saved to {images_root}/batch_{batch_index}')

{'filepath': 'images/batch_0/2024-02-01_16-41-42--SYEW0373.JPG', 'latitude': 41.906782, 'longitude': -87.651927}
CPU times: user 4min 1s, sys: 2.28 s, total: 4min 3s
Wall time: 1min 58s
Predictions:
images/batch_0/2024-02-01_16-41-42--SYEW0373.JPG => f1856211-cfb7-4a5b-9158-c0f72fd09ee6;;;;;;blank
predictions_dict_0.json saved to images/batch_0
{'filepath': 'images/batch_1/2024-02-01_16-41-42--SYEW1833.JPG', 'latitude': 41.906782, 'longitude': -87.651927}
CPU times: user 4min 22s, sys: 2.14 s, total: 4min 24s
Wall time: 2min 8s
Predictions:
images/batch_1/2024-02-01_16-41-42--SYEW1833.JPG => f1856211-cfb7-4a5b-9158-c0f72fd09ee6;;;;;;blank
predictions_dict_1.json saved to images/batch_1
{'filepath': 'images/batch_2/2024-02-01_Bubbly_spypoint_garden--HDPH0058.JPG', 'latitude': 41.906782, 'longitude': -87.651927}
CPU times: user 3min 38s, sys: 2.29 s, total: 3min 41s
Wall time: 1min 47s
Predictions:
images/batch_2/2024-02-01_Bubbly_spypoint_garden--HDPH0058.JPG => f1856211-cfb7-4a5b-9158-

In [29]:
# To concatenate all the json files
output_file = output_root / "predictions_dict_master.json"

# Initialize the master predictions list
master_predictions = []

# Loop through files matching the pattern
for json_file in sorted(images_root.glob("batch_*/predictions_dict_*.json")):
    with open(json_file, "r") as f:
        data = json.load(f)
        if "predictions" in data:
            master_predictions.extend(data["predictions"])  # Concatenate predictions!
        else:
            print(f"{json_file} missing 'predictions' key")

# Write the combined predictions to a new file
with open(output_file, "w") as f:
    json.dump({"predictions": master_predictions}, f, indent=2)

print(f"Combined {len(master_predictions)} predictions into {output_file}")


Combined 11128 predictions into output/predictions_dict_master.json


## Let's save the predictions dict json file

In [ ]:
# Create a docs folder for previewing the images
output_path = '/kaggle/working/output/docs'
os.makedirs(output_path, exist_ok=True)

!python -m megadetector.postprocessing.postprocess_batch_results /kaggle/working/output/predictions_dict_master.json /kaggle/working/output/docs --num_images_to_sample -1 --confidence_threshold 0.5

Loading results from /kaggle/working/output/predictions_dict_master.json
This appears to be a SpeciesNet output file, converting to MD format
Writing temporary results to /tmp/megadetector_temp_files/663ee936-34ce-11f0-b1bd-0242ac130202.json
Converting results to dataframe
Finished loading MegaDetector results for 11128 images from /kaggle/working/output/predictions_dict_master.json
Assigning images to rendering categories
100%|██████████████████████████████████| 11128/11128 [00:00<00:00, 27840.67it/s]
Finished loading and preprocessing 11128 rows from detector output, predicted 499 positives.
 43%|████████████████▍                     | 4802/11128 [00:55<01:06, 94.68it/s]

## Let's zip the folder so we can easily download it

In [31]:
shutil.make_archive('/kaggle/working/output/docs', 'zip', '/kaggle/working/output/docs')

# and finally clean up the tree that made it this far
shutil.rmtree('/kaggle/working/output/docs')  # Deletes the folder
